In [1]:
# import packages
import pandas as pd

In [2]:
metadata = pd.read_csv("/mnt/c/Users/guzh/tax_data/bac120_metadata.tsv", sep='\t')

/tmp/ipykernel_11656/422000499.py:1: DtypeWarning: Columns (61,63,65,74,82,83) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("/mnt/c/Users/guzh/tax_data/bac120_metadata.tsv", sep='\t')


In [5]:
for i, j in zip(metadata.columns, metadata.iloc[0]):
    print(i, j)

accession RS_GCF_001206855.1
ambiguous_bases 14
checkm_completeness 99.6
checkm_contamination 0.91
checkm_marker_count 761
checkm_marker_lineage g__Streptococcus (UID584)
checkm_marker_set_count 238
checkm_strain_heterogeneity 50.0
coding_bases 1827337
coding_density 86.24672681585024
contig_count 77
gc_count 837175
gc_percentage 39.513672790399006
genome_size 2118732
gtdb_genome_representative RS_GCF_001457635.1
gtdb_representative f
gtdb_taxonomy d__Bacteria;p__Bacillota;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus pneumoniae
gtdb_type_designation_ncbi_taxa not type material
gtdb_type_designation_ncbi_taxa_sources none
gtdb_type_species_of_genus f
l50_contigs 13
l50_scaffolds 13
longest_contig 119381
longest_scaffold 119381
lsu_23s_contig_len 45341
lsu_23s_count 1
lsu_23s_length 2898
lsu_23s_query_id NZ_CRLU01000016.1
lsu_5s_contig_len 45341
lsu_5s_count 1
lsu_5s_length 109
lsu_5s_query_id NZ_CRLU01000016.1
lsu_silva_23s_blast_align_len 2898
lsu

In [6]:
import re

def get_level(gtdb_taxonomy_string, pattern):
    match = re.search(pattern, gtdb_taxonomy_string)

    if match:
        # Extract and print the matched group
        name = match.group(1)
        return name
    else:
        return None

genus_name = []
family_name = []
species_name = []

genus_pattern = r'g__([^;]+)'
family_pattern = r'f__([^;]+)'
species_pattern = r's__([^;]+)'

for t in metadata["gtdb_taxonomy"]:
    genus_name.append(get_level(t, genus_pattern))
    family_name.append(get_level(t, family_pattern))
    species_name.append(get_level(t, species_pattern))


metadata["genus_name"] = genus_name
metadata["family_name"] = family_name
metadata["species_name"] = species_name

In [20]:
metadata

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,ssu_silva_blast_perc_identity,ssu_silva_blast_subject_id,ssu_silva_taxonomy,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count,genus_name,family_name,species_name
0,RS_GCF_001206855.1,14,99.60,0.91,761,g__Streptococcus (UID584),238,50.00,1827337,86.246727,...,99.935,CP028436.1781495.1783046,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,21,19,35,0,Streptococcus,Streptococcaceae,Streptococcus pneumoniae
1,RS_GCF_001108865.1,13,99.39,0.34,761,g__Streptococcus (UID584),238,33.33,1852015,86.277715,...,99.935,AKVY01000001.1976120.1977665,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,0,19,38,0,Streptococcus,Streptococcaceae,Streptococcus pneumoniae
2,RS_GCF_000730125.1,30,100.00,0.92,476,f__Bifidobacteriaceae (UID1462),217,0.00,2212698,86.271453,...,100,CP008885.1576966.1578495,Bacteria;Actinobacteriota;Actinobacteria;Bifid...,619,19,55,0,Bifidobacterium,Bifidobacteriaceae,Bifidobacterium infantis
3,RS_GCF_000950275.1,0,99.45,0.53,921,f__Vibrionaceae (UID4865),366,0.00,4128151,84.994806,...,100,KY272023.1.1437,Bacteria;Proteobacteria;Gammaproteobacteria;En...,0,17,77,1,Photobacterium,Vibrionaceae,Photobacterium angustum
4,RS_GCF_000468955.1,0,100.00,0.00,472,o__Lactobacillales (UID543),265,0.00,2107260,86.823994,...,99.935,CP003157.2105633.2107180,Bacteria;Firmicutes;Bacilli;Lactobacillales;St...,0,19,60,0,Lactococcus,Streptococcaceae,Lactococcus cremoris
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394927,GB_GCA_934776845.1,0,91.57,0.00,149,k__Bacteria (UID2329),89,0.00,779596,89.630474,...,87.047,FJ951868.1.1481,Bacteria;Firmicutes;Clostridia;Christensenella...,0,19,41,0,UMGS687,UBA1242,UMGS687 sp900544595
394928,GB_GCA_023647245.1,4609,99.97,0.13,1162,f__Enterobacteriaceae (UID5121),336,0.00,4707023,87.866887,...,99.316,CP033396.457925.459474,Bacteria;Proteobacteria;Gammaproteobacteria;En...,0,19,80,1,Klebsiella,Enterobacteriaceae,Klebsiella pneumoniae
394929,GB_GCA_020344015.1,0,93.69,3.30,147,k__Bacteria (UID2495),91,0.00,4223673,87.882712,...,none,none,none,27200,18,46,1,G020344215,KS3-K002,G020344215 sp020344015
394930,GB_GCA_945889495.1,0,90.94,3.23,247,c__Deltaproteobacteria (UID3216),155,0.00,4966708,87.941933,...,98.57,FPLL01004645.16.1553,Bacteria;RCP2-54;metagenome,0,18,46,1,DP-20,UBA9968,DP-20 sp945889495


In [21]:
sample_df = metadata[["ncbi_genbank_assembly_accession", "family_name", "genus_name", "species_name"]]

In [22]:
sample_df.to_csv("metadata.csv")

In [27]:
from ete4 import NCBITaxa

ncbi = NCBITaxa()

In [25]:
def get_level_id(taxid, level='genus'):
    """
    Get the taxid of the genus given a species taxid.
    """
    for i in ncbi.get_lineage(taxid):
        rank = ncbi.get_rank([i])[i]
        if rank == level:
            return i
    return None

In [28]:
from tqdm import tqdm

genus_taxid = []
species_taxid = []
family_taxid = []

for i in tqdm(metadata["ncbi_species_taxid"]):
    genus_taxid.append(get_level_id(i, 'genus'))
    species_taxid.append(get_level_id(i, 'species'))
    family_taxid.append(get_level_id(i, 'family'))


metadata["genus_taxid"] = genus_taxid
metadata["species_taxid"] = species_taxid
metadata["family_taxid"] = family_taxid

  0%|          | 171/394932 [00:00<07:17, 901.49it/s]/home/zhenhao/.local/lib/python3.10/site-packages/ete4/ncbi_taxonomy/ncbiquery.py:216: UserWarning: taxid 1313265 was translated into 1302351
  warnings.warn('taxid %s was translated into %s' %
  0%|          | 261/394932 [00:00<08:01, 820.11it/s]/home/zhenhao/.local/lib/python3.10/site-packages/ete4/ncbi_taxonomy/ncbiquery.py:216: UserWarning: taxid 1217702 was translated into 70348
  warnings.warn('taxid %s was translated into %s' %
  0%|          | 1050/394932 [00:01<05:56, 1103.69it/s]/home/zhenhao/.local/lib/python3.10/site-packages/ete4/ncbi_taxonomy/ncbiquery.py:216: UserWarning: taxid 92835 was translated into 69369
  warnings.warn('taxid %s was translated into %s' %
  1%|          | 3560/394932 [00:02<05:01, 1297.40it/s]/home/zhenhao/.local/lib/python3.10/site-packages/ete4/ncbi_taxonomy/ncbiquery.py:216: UserWarning: taxid 1217712 was translated into 1776742
  warnings.warn('taxid %s was translated into %s' %
  1%|         

In [14]:
import numpy as np

metadata.groupby("genus_taxid").count().sort_values(by=['accession'])

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,ssu_silva_blast_evalue,ssu_silva_blast_perc_identity,ssu_silva_blast_subject_id,ssu_silva_taxonomy,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count,ncbi_genus_taxid,species_taxid
genus_taxid,,,,,,,,,,,,,,,,,,,,,
3143992.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2066071.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2066435.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2070443.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2070447.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590.0,14117,14117,14117,14117,14117,14117,14117,14117,14117,14117,...,14117,14117,14117,14117,14117,14117,14117,14117,14117,14117
570.0,17915,17915,17915,17915,17915,17915,17915,17915,17915,17915,...,17915,17915,17915,17915,17915,17915,17915,17915,17915,17915
1301.0,18437,18437,18437,18437,18437,18437,18437,18437,18437,18437,...,18437,18437,18437,18437,18437,18437,18437,18437,18437,18437


In [15]:
escherichia_df = metadata[metadata["genus_taxid"] == 561].groupby("species_taxid").sample(1)

In [39]:
staphylococcus_df = metadata[metadata["genus_taxid"] == 1279].groupby("species_taxid").sample(1)

In [32]:
escherichia_df.iloc[0]["ncbi_genbank_assembly_accession"]

'GCA_000618325.1'

In [41]:
import subprocess
import os

os.chdir("/home/zhenhao/TDT/data")

os.mkdir("staphylococcus")
os.chdir("staphylococcus")


In [ ]:
import glob

for i in tqdm(range(len(staphylococcus_df))):
    accession = staphylococcus_df.iloc[i]["ncbi_genbank_assembly_accession"]
    taxid = str(staphylococcus_df.iloc[i]["species_taxid"])

    subprocess.run(["datasets", "download", "genome", "accession", accession, "--no-progressbar", "--filename", taxid + ".zip"])
    if os.path.exists(taxid + ".zip"):
        subprocess.run(["unzip", taxid + ".zip"])

        files_name = glob.glob("ncbi_dataset/data/*/*.fna")
        with open(taxid + ".fna", "wb") as f:
            subprocess.run(["cat"] + files_name, stdout=f)
        subprocess.run(["rm", "-r", "ncbi_dataset", taxid + ".zip", "README.md"])

In [45]:
metadata[["ncbi_genbank_assembly_accession", "genus_taxid", "species_taxid"]].to_csv("metadata.csv")

In [50]:
len(set(metadata["genus_taxid"]))

89851

In [52]:
set(metadata[metadata["genus_taxid"] != np.nan]["genus_taxid"])

{nan,
 nan,
 6.0,
 nan,
 nan,
 10.0,
 nan,
 13.0,
 nan,
 nan,
 16.0,
 nan,
 18.0,
 nan,
 20.0,
 nan,
 22.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 32.0,
 nan,
 nan,
 nan,
 nan,
 40.0,
 nan,
 42.0,
 nan,
 44.0,
 nan,
 47.0,
 nan,
 nan,
 50.0,
 nan,
 53.0,
 nan,
 55.0,
 nan,
 nan,
 59.0,
 nan,
 nan,
 nan,
 64.0,
 nan,
 nan,
 68.0,
 nan,
 71.0,
 nan,
 nan,
 75.0,
 nan,
 nan,
 nan,
 81.0,
 nan,
 nan,
 85.0,
 nan,
 nan,
 88.0,
 nan,
 nan,
 93.0,
 nan,
 nan,
 96.0,
 nan,
 99.0,
 nan,
 101.0,
 nan,
 nan,
 105.0,
 nan,
 107.0,
 nan,
 nan,
 nan,
 113.0,
 nan,
 nan,
 nan,
 118.0,
 nan,
 nan,
 123.0,
 nan,
 nan,
 127.0,
 nan,
 nan,
 nan,
 133.0,
 nan,
 nan,
 nan,
 138.0,
 nan,
 nan,
 nan,
 nan,
 146.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 157.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 171.0,
 nan,
 nan,
 nan,
 nan,
 179.0,
 nan,
 1048757.0,
 nan,
 nan,
 nan,
 187.0,
 nan,
 nan,
 191.0,
 nan,
 nan,
 194.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 209.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 222.0,
 na

In [57]:
set(metadata.dropna(subset=['genus_taxid'])["genus_taxid"])

{1548290.0,
 1654787.0,
 6.0,
 131079.0,
 286729.0,
 1843210.0,
 10.0,
 3047433.0,
 13.0,
 65551.0,
 16.0,
 18.0,
 20.0,
 22.0,
 1400859.0,
 1892380.0,
 32.0,
 49184.0,
 2752546.0,
 2752545.0,
 1906657.0,
 40.0,
 42.0,
 44.0,
 47.0,
 2056239.0,
 1187887.0,
 73778.0,
 50.0,
 1906659.0,
 53.0,
 1384499.0,
 335927.0,
 55.0,
 59.0,
 401469.0,
 442430.0,
 442431.0,
 999488.0,
 64.0,
 393277.0,
 1458243.0,
 68.0,
 1343556.0,
 139331.0,
 71.0,
 75.0,
 286800.0,
 81.0,
 999506.0,
 286801.0,
 2490452.0,
 85.0,
 360528.0,
 458839.0,
 88.0,
 1679449.0,
 622681.0,
 1777753.0,
 106589.0,
 483422.0,
 106591.0,
 93.0,
 96.0,
 475230.0,
 99.0,
 2678884.0,
 2678885.0,
 221279.0,
 221282.0,
 101.0,
 105.0,
 2678886.0,
 107.0,
 3031148.0,
 2981995.0,
 221285.0,
 221292.0,
 1237104.0,
 2777201.0,
 113.0,
 311411.0,
 245876.0,
 688241.0,
 118.0,
 2951993.0,
 303226.0,
 123.0,
 49277.0,
 49279.0,
 127.0,
 90243.0,
 133.0,
 57479.0,
 1204360.0,
 106633.0,
 138.0,
 98443.0,
 2048137.0,
 3063947.0,
 467084.0,


In [62]:
metadata[metadata["genus_taxid"] == 6].groupby("species_taxid").sample(1)

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,ssu_silva_blast_perc_identity,ssu_silva_blast_subject_id,ssu_silva_taxonomy,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count,ncbi_genus_taxid,genus_taxid,species_taxid
98606,RS_GCF_000010525.1,0,100.00,0.00,485,o__Rhizobiales (UID3642),316,0.0,4769667,88.824386,...,100,AP009384.5045775.5047242,Bacteria;Proteobacteria;Alphaproteobacteria;Rh...,0,19,53,1,6.0,6.0,7
4560,RS_GCF_000473085.1,111,99.68,0.00,485,o__Rhizobiales (UID3642),316,0.0,5147322,88.479753,...,99.861,DQ664248.1.1434,Bacteria;Proteobacteria;Alphaproteobacteria;Rh...,1611,19,49,1,6.0,6.0,281091
60844,RS_GCF_014635325.1,0,99.68,0.63,485,o__Rhizobiales (UID3642),316,0.0,5600050,87.549672,...,99.758,FR799325.1.1443,Bacteria;Proteobacteria;Alphaproteobacteria;Rh...,128,19,50,1,6.0,6.0,980631
322618,GB_GCA_002280795.1,62,84.41,1.42,485,o__Rhizobiales (UID3642),316,50.0,3473741,88.622941,...,none,none,none,4034,17,32,1,6.0,6.0,1970319
337638,RS_GCF_004364705.1,0,100.00,0.00,485,o__Rhizobiales (UID3642),316,0.0,4873389,89.014858,...,100,AP009384.5045775.5047242,Bacteria;Proteobacteria;Alphaproteobacteria;Rh...,93,19,49,1,6.0,6.0,2183897


In [23]:
# Use a subset of genera

labels_file = "/mnt/c/Users/guzh/Downloads/sampled_reads.label"
read_species_taxid = []
with open(labels_file, "r") as f:
    for line in f.readlines():
        read_species_taxid.append(int(line.strip()))

In [69]:
read_species_taxid

[100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 100886,
 

In [37]:
read_genus_taxid = []
for i in set(read_species_taxid):
    read_genus_taxid.append(get_level_id(i, 'family'))

In [38]:
read_genus_taxid = [g for g in read_genus_taxid if g is not None]

In [43]:
read_genus_taxid

[1843491,
 128827,
 194924,
 2005525,
 815,
 1643826,
 31953,
 815,
 815,
 1643826,
 2810280,
 31953,
 31977,
 815,
 815,
 186803,
 815,
 2005525,
 909930,
 31977,
 186803,
 2810281,
 815,
 815,
 186804,
 815,
 815,
 216572,
 84107,
 1643824,
 186803,
 31979,
 1647988,
 186803,
 31977,
 186803,
 186803,
 186803,
 216572,
 128827,
 31977,
 186803,
 1300,
 186803,
 216572,
 815,
 171550,
 1853231,
 815,
 216572,
 31979,
 186803,
 31979,
 31979,
 31953,
 31979,
 31979,
 186803,
 216572,
 815,
 31953,
 186804,
 128827,
 194924,
 815,
 216572]

In [47]:
for i in ncbi.get_lineage(216572):
    rank = ncbi.get_rank([i])[i]
    print(i, rank)

1 no rank
131567 no rank
2 superkingdom
1783272 clade
1239 phylum
186801 class
186802 order
216572 family


In [41]:
len(set(read_genus_taxid))

21

In [42]:
metadata[metadata["family_taxid"].isin(set(read_genus_taxid))].to_csv("/home/zhenhao/TDT/gtdb_utils/metadata_sample.csv")

In [36]:
len(read_genus_taxid)

66

In [34]:
sample_df

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count,genus_name,family_name,species_name,genus_taxid,species_taxid,family_taxid
2,RS_GCF_000730125.1,30,100.00,0.92,476,f__Bifidobacteriaceae (UID1462),217,0.0,2212698,86.271453,...,619,19,55,0,Bifidobacterium,Bifidobacteriaceae,Bifidobacterium infantis,1678.0,216816,31953.0
4,RS_GCF_000468955.1,0,100.00,0.00,472,o__Lactobacillales (UID543),265,0.0,2107260,86.823994,...,0,19,60,0,Lactococcus,Streptococcaceae,Lactococcus cremoris,1357.0,1359,1300.0
22,RS_GCF_001078445.1,7,99.32,0.00,340,o__Clostridiales (UID1342),177,0.0,4662609,85.403668,...,32463,19,77,1,Enterocloster,Lachnospiraceae,Enterocloster clostridioformis,2719313.0,1531,186803.0
44,RS_GCF_000273155.1,0,99.26,0.00,492,o__Bacteroidales (UID2657),269,0.0,4834627,88.797615,...,14319,19,83,0,Bacteroides,Bacteroidaceae,Bacteroides fragilis,816.0,817,815.0
50,RS_GCF_000009925.1,0,99.26,0.00,492,o__Bacteroidales (UID2657),269,0.0,4719009,88.853660,...,0,19,74,0,Bacteroides,Bacteroidaceae,Bacteroides fragilis,816.0,817,815.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394902,GB_GCA_930989685.1,0,97.96,0.68,261,o__Clostridiales (UID1212),147,0.0,1974955,87.497032,...,0,18,41,0,Ruthenibacterium,Ruminococcaceae,Ruthenibacterium merdavium,216851.0,259315,216572.0
394904,GB_GCA_934668345.1,0,68.55,0.00,211,p__Actinobacteria (UID2112),124,0.0,1143151,87.849141,...,0,18,40,1,Collinsella,Coriobacteriaceae,Collinsella sp934668345,102106.0,165190,84107.0
394914,GB_GCA_945900725.1,0,92.79,0.68,229,k__Bacteria (UID2982),147,0.0,3994946,89.692739,...,0,18,37,1,CAJBME01,V1-33,CAJBME01 sp903954065,239934.0,239935,1647988.0
394919,GB_GCA_021625385.1,0,80.87,0.89,263,o__Clostridiales (UID1212),149,50.0,1358322,88.332645,...,310,9,16,0,Vescimonas,Oscillospiraceae,Vescimonas sp900556015,459786.0,1945593,216572.0
